In [1]:
import librosa
import glob
import os
import dtw
import matplotlib.pyplot as plt
import seaborn
seaborn.set(style='dark')
%matplotlib inline

import msaf # develop branch
from msaf import jams2
import numpy as np
import scipy
import mir_eval
import scipy.spatial.distance
from joblib import Parallel, delayed
import cPickle as pickle

	python setup.py build_ext --inplace


In [2]:
ds_path = "/Users/uri/datasets/Segments/"
ds_path = "/home/uri/datasets/Segments/"

In [3]:
%run 'Cross-repetition DTW.ipynb'

:0: FutureWarning: IPython widgets are experimental and may change in the future.


IndexError: list index out of range

In [4]:
%run 'Repetition detection threshold.ipynb'

Variable           Type        Data/Info
----------------------------------------
Parallel           type        <class 'joblib.parallel.Parallel'>
delayed            function    <function delayed at 0x7f7891da6758>
dist               str         L1
ds_path            str         /home/uri/datasets/Segments/
dtw                module      <module 'dtw' from '/usr/<...>7/dist-packages/dtw.pyc'>
file_scores        list        n=769
glob               module      <module 'glob' from '/usr<...>/lib/python2.7/glob.pyc'>
interact           function    <function interact at 0x7f7875ec4f50>
jams2              module      <module 'msaf.jams2' from<...>msaf/jams2/__init__.pyc'>
k                  int         39
lab_files          list        n=0
lab_to_aud         function    <function lab_to_aud at 0x7f7875ed00c8>
level              str         large_scale
librosa            module      <module 'librosa' from '/<...>gg/librosa/__init__.pyc'>
make_cost_matrix   function    <function make_cost_ma

TypeError: plot_curve() got an unexpected keyword argument 'paths'

## Parameters to Explore ##

Datasets (levels):

* SALAMI (large_scale, small_scale)
* Isophonics (function)

Distances for the DTW scores:

* L1
* L2
* correlation

Normalization of the DTW scores to compute the threshold and the F-measures:

* None
* Minimum
* Maximum
* Harmonic Mean

In [5]:
datasets = ["SALAMI", "Isophonics"]
dist_dict = {
    "L1": scipy.spatial.distance.cityblock, 
    "L2": scipy.spatial.distance.euclidean,
    "correlation": scipy.spatial.distance.correlation
}
dataset_levels = {
    "Isophonics": ["function"],
    "SALAMI": ["large_scale", "small_scale"]
}
norms = ["none", "min", "max", "hmean"]

In [6]:
def compute_score(file_struct, level, dist_key):
    try:
        ref_inter, ref_labels = jams2.converters.load_jams_range(
            file_struct.ref_file, "sections", annotator=0, context=level)
        D, P = make_cost_matrix(file_struct.audio_file, ref_inter, ref_labels, dist=dist_dict[dist_key])
        thresholds = {}
        fmeasures = {}
        for norm in norms:
            thresholds[norm], fmeasures[norm] = plot_curve(file_name=file_struct.audio_file, 
                                                           intervals=ref_inter, labels=ref_labels,
                                                           scores=D, norm=norm)
    except:
        print "warning: no annotations for file", file_struct.audio_file
        ref_inter = None
        ref_labels = None
        D = None
        P = None
        thresholds = None
        fmeasures = None
    ret = {
        "intervals": ref_inter,
        "labels": ref_labels,
        "scores": D,
        "paths": P,
        "thresholds": thresholds,
        "fmeasures": fmeasures,
        "file_name": os.path.basename(file_struct.audio_file)
    }
    return ret

In [7]:
n_jobs = 8

for dataset in datasets:
    files = msaf.io.get_dataset_files(ds_path, ds_name=dataset)
    for level in dataset_levels[dataset]:
        for dist_key in dist_dict.keys():
            print "Computing: ", dataset, level, dist_key
            scores = Parallel(n_jobs=n_jobs)(delayed(compute_score)(
                file_struct, level, dist_key)
                for file_struct in files[:])
            result = {
                "level": level,
                "dist": dist_key,
                "file_scores": scores
            }
            out_file = "scores_datasetE%s_levelE%s_distE%s.pk" % (dataset, level, dist_key)
            with open(out_file, "w") as f:
                pickle.dump(result, f, protocol=-1)

Computing:  SALAMI large_scale correlation
Computing:  SALAMI large_scale L2
 /home/uri/datasets/Segments/audio/SALAMI_960.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1562.mp3
 /home/uri/datasets/Segments/audio/SALAMI_486.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1180.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1462.mp3
 /home/uri/datasets/Segments/audio/SALAMI_540.mp3
 /home/uri/datasets/Segments/audio/SALAMI_842.mp3
 /home/uri/datasets/Segments/audio/SALAMI_356.mp3
 /home/uri/datasets/Segments/audio/SALAMI_970.mp3
 /home/uri/datasets/Segments/audio/SALAMI_40.mp3
 /home/uri/datasets/Segments/audio/SALAMI_304.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1258.mp3
 /home/uri/datasets/Segments/audio/SALAMI_772.mp3
 /home/uri/datasets/Segments/audio/SALAMI_12.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1390.mp3
 /home/uri/datasets/Segments/audio/SALAMI_866.mp3
 /home/uri/datasets/Segments/audio/SALAMI_362.mp3
 /home/uri/datasets/Segments/audio/SALAMI_634.mp3
 /home/uri/datasets/

/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-pa

 /home/uri/datasets/Segments/audio/SALAMI_270.mp3
 /home/uri/datasets/Segments/audio/SALAMI_168.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1310.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1484.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1010.mp3
 /home/uri/datasets/Segments/audio/SALAMI_578.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1160.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1202.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1432.mp3
 /home/uri/datasets/Segments/audio/SALAMI_72.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1358.mp3
 /home/uri/datasets/Segments/audio/SALAMI_120.mp3
 /home/uri/datasets/Segments/audio/SALAMI_536.mp3
 /home/uri/datasets/Segments/audio/SALAMI_816.mp3
 /home/uri/datasets/Segments/audio/SALAMI_456.mp3
 /home/uri/datasets/Segments/audio/SALAMI_562.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1002.mp3
 /home/uri/datasets/Segments/audio/SALAMI_676.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1038.mp3
 /home/uri/datasets/Segments/audio/SALAMI_

/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-pa

 /home/uri/datasets/Segments/audio/SALAMI_1506.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1060.mp3
 /home/uri/datasets/Segments/audio/SALAMI_806.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1432.mp3
 /home/uri/datasets/Segments/audio/SALAMI_516.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1530.mp3
 /home/uri/datasets/Segments/audio/SALAMI_816.mp3
 /home/uri/datasets/Segments/audio/SALAMI_360.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1010.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1002.mp3
 /home/uri/datasets/Segments/audio/SALAMI_88.mp3
 /home/uri/datasets/Segments/audio/SALAMI_386.mp3
 /home/uri/datasets/Segments/audio/SALAMI_562.mp3
 /home/uri/datasets/Segments/audio/SALAMI_294.mp3
 /home/uri/datasets/Segments/audio/SALAMI_886.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1038.mp3
 /home/uri/datasets/Segments/audio/SALAMI_544.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1514.mp3
 /home/uri/datasets/Segments/audio/SALAMI_132.mp3
 /home/uri/datasets/Segments/audio/SALAMI_5

...

/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-pa

 /home/uri/datasets/Segments/audio/SALAMI_636.mp3
 /home/uri/datasets/Segments/audio/SALAMI_738.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1514.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1082.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1498.mp3
 /home/uri/datasets/Segments/audio/SALAMI_770.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1058.mp3
 /home/uri/datasets/Segments/audio/SALAMI_436.mp3
 /home/uri/datasets/Segments/audio/SALAMI_520.mp3
 /home/uri/datasets/Segments/audio/SALAMI_806.mp3
 /home/uri/datasets/Segments/audio/SALAMI_690.mp3
 /home/uri/datasets/Segments/audio/SALAMI_346.mp3
 /home/uri/datasets/Segments/audio/SALAMI_578.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1342.mp3
 /home/uri/datasets/Segments/audio/SALAMI_380.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1208.mp3
 /home/uri/datasets/Segments/audio/SALAMI_916.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1358.mp3
 /home/uri/datasets/Segments/audio/SALAMI_72.mp3
 /home/uri/datasets/Segments/audio/SALAMI_48

/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:706: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  np.all(classes == [-1, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:705: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  not (np.all(classes == [0, 1]) or
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.15.2-py2.7-linux-x86_64.egg/sklearn/metrics/metrics.py:706: DeprecationWarning: elementwise comparison failed; this will raise the error in the future.
  np.all(classes == [-1, 1]) or


KeyboardInterrupt: 

 /home/uri/datasets/Segments/audio/SALAMI_1342.mp3
 /home/uri/datasets/Segments/audio/SALAMI_188.mp3
 /home/uri/datasets/Segments/audio/SALAMI_120.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1530.mp3
 /home/uri/datasets/Segments/audio/SALAMI_938.mp3
 /home/uri/datasets/Segments/audio/SALAMI_336.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1432.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1060.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1032.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1344.mp3
 /home/uri/datasets/Segments/audio/SALAMI_516.mp3
 /home/uri/datasets/Segments/audio/SALAMI_816.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1310.mp3
 /home/uri/datasets/Segments/audio/SALAMI_690.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1002.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1202.mp3
 /home/uri/datasets/Segments/audio/SALAMI_1208.mp3
 /home/uri/datasets/Segments/audio/SALAMI_562.mp3
 /home/uri/datasets/Segments/audio/SALAMI_536.mp3
 /home/uri/datasets/Segments/audio/SALAM

In [27]:
# Remove redundant data
with open("scores_datasetESALAMI_levelElarge_scale_distEL1.pk", "r") as f:
    data = pickle.load(f)

In [41]:
pk_files = glob.glob("*.pk")
pk_files = ["scores_datasetESALAMI_levelEsmall_scale_distEL1.pk"]
for pk_file in pk_files:
    with open(pk_file, "r") as f:
        data = pickle.load(f)
    updated_scores = []
    for file_scores in data["file_scores"]:
        new_entry = file_scores
        new_entry["scores"] = np.asarray(file_scores["scores"], dtype=np.float32)
        if file_scores["labels"] is not None:
            N = len(file_scores["labels"])
            for i in range(N):
                for j in range(N):
                    if j <= i:
                        # Remove lower triangle
                        new_entry["paths"][i][j] = 0
                    else:
                        # Convert to int32
                        try:
                            if len(new_entry["paths"][i][j]) == 2:
                                new_entry["paths"][i][j][0] = np.asarray(new_entry["paths"][i][j][0], dtype=np.int32)
                                new_entry["paths"][i][j][1] = np.asarray(new_entry["paths"][i][j][1], dtype=np.int32)
                        except:
                            pass
        updated_scores.append(new_entry)

    data["file_scores"] = updated_scores
    with open("conv_" + pk_file, "w") as f:
        data = pickle.dump(data, f, protocol=-1)

In [18]:
caca = np.asarray([1,2,3,4], dtype=np.int32)
data["file_scores"][0]["scores"].dtype

dtype('float64')